In [8]:
import heapq
import time
from collections import deque
import math
# Definición de las funciones a utilizar
def cargar_laberinto(nombre_archivo):
    with open(nombre_archivo, 'r') as file:
        laberinto = [line.strip().split(',') for line in file]
    return laberinto

def encontrar_posiciones(laberinto, valor):
    posiciones = []
    for fila in range(len(laberinto)):
        for col in range(len(laberinto[0])):
            if laberinto[fila][col] == valor:
                posiciones.append((fila, col))
    return posiciones

# Definición de las distancias
def manhattan(a, b):
    return abs(a[0] - b[0]) + abs(a[1] - b[1])

def euclidiana(a, b):
    return math.sqrt((a[0] - b[0]) ** 2 + (a[1] - b[1]) ** 2)

def movimientos_validos(laberinto, pos):
    filas, columnas = len(laberinto), len(laberinto[0])
    movs = [(0, -1), (1, 0), (0, 1), (-1, 0)]  # Movimientos: izquierda, abajo, derecha, arriba
    vecinos = []
    for dx, dy in movs:
        nx, ny = pos[0] + dx, pos[1] + dy
        if 0 <= nx < filas and 0 <= ny < columnas and laberinto[nx][ny] != '1':
            vecinos.append((nx, ny))
    return vecinos

def reconstruir_camino(padres, inicio, fin):
    camino = []
    actual = fin
    while actual != inicio:
        camino.append(actual)
        actual = padres[actual]
    camino.append(inicio)
    return camino[::-1]

# Definir los algoritmos
def bfs(laberinto, inicio, fin):
    cola = deque([inicio])
    padres = {inicio: None}
    explorados = 0
    while cola:
        nodo = cola.popleft()
        explorados += 1
        if nodo == fin:
            return reconstruir_camino(padres, inicio, fin), explorados
        for vecino in movimientos_validos(laberinto, nodo):
            if vecino not in padres:
                padres[vecino] = nodo
                cola.append(vecino)
    return [], explorados

def dfs(laberinto, inicio, fin):
    pila = [inicio]
    padres = {inicio: None}
    explorados = 0
    while pila:
        nodo = pila.pop()
        explorados += 1
        if nodo == fin:
            return reconstruir_camino(padres, inicio, fin), explorados
        for vecino in movimientos_validos(laberinto, nodo):
            if vecino not in padres:
                padres[vecino] = nodo
                pila.append(vecino)
    return [], explorados

def greedy(laberinto, inicio, fin, heuristica):
    heap = [(heuristica(inicio, fin), inicio)]
    padres = {inicio: None}
    explorados = 0
    while heap:
        _, nodo = heapq.heappop(heap)
        explorados += 1
        if nodo == fin:
            return reconstruir_camino(padres, inicio, fin), explorados
        for vecino in movimientos_validos(laberinto, nodo):
            if vecino not in padres:
                padres[vecino] = nodo
                heapq.heappush(heap, (heuristica(vecino, fin), vecino))
    return [], explorados

def a_estrella(laberinto, inicio, fin, heuristica):
    heap = [(0, inicio)]
    g_score = {inicio: 0}
    padres = {inicio: None}
    explorados = 0
    while heap:
        _, nodo = heapq.heappop(heap)
        explorados += 1
        if nodo == fin:
            return reconstruir_camino(padres, inicio, fin), explorados
        for vecino in movimientos_validos(laberinto, nodo):
            temp_g = g_score[nodo] + 1
            if vecino not in g_score or temp_g < g_score[vecino]:
                g_score[vecino] = temp_g
                f_score = temp_g + heuristica(vecino, fin)
                padres[vecino] = nodo
                heapq.heappush(heap, (f_score, vecino))
    return [], explorados

# Juntar los algoritmos con las distancias
def ejecutar_algoritmos(laberinto, inicio, fin):
    metodos = {
        "Algoritmo BFS": bfs,
        "Algoritmo DFS": dfs,
        "Algoritmo Greedy con distancia de Manhattan": lambda l, i, f: greedy(l, i, f, manhattan),
        "Algoritmo Greedy con distancia Euclidiana": lambda l, i, f: greedy(l, i, f, euclidiana),
        "Algoritmo A* con distancia de Manhattan": lambda l, i, f: a_estrella(l, i, f, manhattan),
        "Algoritmo A* con distancia Euclidiana": lambda l, i, f: a_estrella(l, i, f, euclidiana)
    }
    
    for nombre, metodo in metodos.items():
        inicio_tiempo = time.time()
        camino, explorados = metodo(laberinto, inicio, fin)
        tiempo_ejecucion = time.time() - inicio_tiempo
        print(f"\n{nombre}")
        print(f"Longitud del camino: {len(camino) if camino else 'No se pudo encontrar un camino que llevara a la solucion'}")
        print(f"Cantidad de nodos explorados: {explorados}")
        print(f"Se tardó {tiempo_ejecucion:.6f} segundos")


In [9]:
# Laberinto 1
def main():
    laberinto = cargar_laberinto("Laberinto1.txt")
    puntos_inicio = encontrar_posiciones(laberinto, '2')
    puntos_salida = encontrar_posiciones(laberinto, '3')
    
    if not puntos_inicio or not puntos_salida:
        print("No se encontraron puntos de inicio o salida en el laberinto.")
        return
    
    print("Información de las soluciones dependiendo del método:")
    ejecutar_algoritmos(laberinto, puntos_inicio[0], puntos_salida[0])
    
if __name__ == "__main__":
    main()

Información de las soluciones dependiendo del método:

Algoritmo BFS
Longitud del camino: 251
Cantidad de nodos explorados: 15876
Se tardó 0.021466 segundos

Algoritmo DFS
Longitud del camino: 8001
Cantidad de nodos explorados: 8001
Se tardó 0.012446 segundos

Algoritmo Greedy con distancia de Manhattan
Longitud del camino: 251
Cantidad de nodos explorados: 251
Se tardó 0.001214 segundos

Algoritmo Greedy con distancia Euclidiana
Longitud del camino: 251
Cantidad de nodos explorados: 251
Se tardó 0.001028 segundos

Algoritmo A* con distancia de Manhattan
Longitud del camino: 251
Cantidad de nodos explorados: 251
Se tardó 0.000000 segundos

Algoritmo A* con distancia Euclidiana
Longitud del camino: 251
Cantidad de nodos explorados: 15627
Se tardó 0.037086 segundos


In [40]:
# Laberinto 2
def main():
    laberinto = cargar_laberinto("Laberinto2.txt")
    puntos_inicio = encontrar_posiciones(laberinto, '2')
    puntos_salida = encontrar_posiciones(laberinto, '3')
    
    if not puntos_inicio or not puntos_salida:
        print("No se encontraron puntos de inicio o salida en el laberinto.")
        return
    
    print("Información de las soluciones dependiendo del método:")
    ejecutar_algoritmos(laberinto, puntos_inicio[0], puntos_salida[0])
    
if __name__ == "__main__":
    main()

Información de las soluciones dependiendo del método:

Algoritmo BFS
Longitud del camino: 251
Cantidad de nodos explorados: 15507
Se tardó 0.046894 segundos

Algoritmo DFS
Longitud del camino: 5023
Cantidad de nodos explorados: 10595
Se tardó 0.022548 segundos

Algoritmo Greedy con distancia de Manhattan
Longitud del camino: 367
Cantidad de nodos explorados: 1309
Se tardó 0.002815 segundos

Algoritmo Greedy con distancia Euclidiana
Longitud del camino: 281
Cantidad de nodos explorados: 436
Se tardó 0.001044 segundos

Algoritmo A* con distancia de Manhattan
Longitud del camino: 251
Cantidad de nodos explorados: 1689
Se tardó 0.006083 segundos

Algoritmo A* con distancia Euclidiana
Longitud del camino: 251
Cantidad de nodos explorados: 13378
Se tardó 0.033160 segundos


In [10]:
# Laberinto 3
def main():
    laberinto = cargar_laberinto("Laberinto3.txt")
    puntos_inicio = encontrar_posiciones(laberinto, '2')
    puntos_salida = encontrar_posiciones(laberinto, '3')
    
    if not puntos_inicio or not puntos_salida:
        print("No se encontraron puntos de inicio o salida en el laberinto.")
        return
    
    print("Información de las soluciones dependiendo del método:")
    ejecutar_algoritmos(laberinto, puntos_inicio[0], puntos_salida[0])
    
if __name__ == "__main__":
    main()

Información de las soluciones dependiendo del método:

Algoritmo BFS
Longitud del camino: 125
Cantidad de nodos explorados: 14625
Se tardó 0.020197 segundos

Algoritmo DFS
Longitud del camino: 2675
Cantidad de nodos explorados: 4879
Se tardó 0.007426 segundos

Algoritmo Greedy con distancia de Manhattan
Longitud del camino: 125
Cantidad de nodos explorados: 125
Se tardó 0.001022 segundos

Algoritmo Greedy con distancia Euclidiana
Longitud del camino: 137
Cantidad de nodos explorados: 152
Se tardó 0.000000 segundos

Algoritmo A* con distancia de Manhattan
Longitud del camino: 125
Cantidad de nodos explorados: 125
Se tardó 0.000000 segundos

Algoritmo A* con distancia Euclidiana
Longitud del camino: 125
Cantidad de nodos explorados: 2790
Se tardó 0.006219 segundos
